In [25]:
import requests

url = "https://www.sustainalytics.com/sustapi/companyratings/getcompanyratings"
payload = {
    "industry": "",
    "rating": "",
    "filter": "",
    "page": 1,
    "pageSize": 10,
    "resourcePackage": "Sustainalytics"
}
headers = {
    "Content-Type": "application/json",
    "User-Agent": "Mozilla/5.0",  # Mimic browser
    # "Authorization": "Bearer YOUR_TOKEN"  # If required
}

# Add session cookies if logged in (extract from browser)
cookies = {
    "_cfuvid": "6z4GGDrXPyUJJ9qDAFNamlvH_XEGHC6I5jB6MCcqGaM-1744803173885-0.0.1.1-604800000; sf-data-intell-subject=1744803174514-5696857f-9a76-45b7-93a7-43b8b6b2105c; sf-prs-ss=638803999754760000; sf-prs-lu=https%3A%2F%2Fwww.sustainalytics.com%2Fesg-data; _zitok=79cf1f55ec9024b768541744803177; sf-ins-ssid=1744971183616-77a622a8-a47a-477c-9122-d6b4a1d63469; sf_abtests=6d9de14d-d722-452d-99af-702de9dbebc1,46ad2ba7-8713-472b-bc2e-2402ac50779f; sf-ins-pv-id=2917c3e9-5f60-4709-8270-f55684359625"
}

response = requests.post(
    url,
    json=payload,
    headers=headers,
    cookies=cookies
)

#print(response.status_code)
#print(response.content)

from bs4 import BeautifulSoup

# Assuming `plain_text_html` contains your HTML
soup = BeautifulSoup(response.content, 'html.parser')

# Find all <a> tags with data-href
anchors = soup.find_all('a', attrs={'data-href': True})

# Extract data-href values
href_values = [a['data-href'] for a in anchors]

print(href_values)
print(f"Found {len(href_values)} data-href links.")


['/1-800-flowers-com-inc/1007902331', '/1-1-ag/1264207944', '/10x-genomics-inc/2000259050', '/11-bit-studios-sa/2002992103', '/1st-source-corp/1008054650', '/2020-bulkers-ltd/2005551813', '/23andme-holding-co/2008929454', '/29metals-ltd/2008918361', '/2i-rete-gas-spa/1008505575', '/2seventy-bio-inc/2008843597']
Found 10 data-href links.


In [24]:
import requests
from bs4 import BeautifulSoup
import time

# Configuration
BASE_URL = "https://www.sustainalytics.com/sustapi/companyratings/getcompanyratings"
OUTPUT_FILE = "sustainalytics_links.txt"
TOTAL_PAGES = 100
PAGE_SIZE = 140

# Headers (update with your actual headers from browser inspection)
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Content-Type": "application/json",
    # Add any required authorization headers here
}

# Session to persist cookies
session = requests.Session()
cookies = {
    "_cfuvid": "Ueg6W8vWZEeMWnWdgym1NUGuVFNcSGvYgk3JNYWB_7A-1745046246825-0.0.1.1-604800000; sf-ins-ssid=1745046247141-77c1ab7b-2767-4557-b9d4-9f802d4df3a4; sf-ins-pv-id=27950fa8-4765-4f76-b8ae-9415c80cdaf7"
}
def fetch_page(page_number):
    """Fetch a single page of results"""
    payload = {
        "industry": "",
        "rating": "",
        "filter": "",
        "page": page_number,
        "pageSize": PAGE_SIZE,
        "resourcePackage": "Sustainalytics"
    }
    
    try:
        response = requests.post(
            BASE_URL,
            json=payload,
            headers=HEADERS,
            cookies=cookies
        )
        #response.raise_for_status()
        return response.content
    except requests.exceptions.RequestException as e:
        print(f"Error fetching page {page_number}: {e}")
        return None

def extract_data_hrefs(html_content):
    """Extract all data-href values from anchor tags"""
    soup = BeautifulSoup(html_content, 'html.parser')
    anchors = soup.find_all('a', attrs={'data-href': True})
    return [a['data-href'] for a in anchors]

def main():
    all_links = []
    
    for page in range(1, TOTAL_PAGES + 1):
        print(f"Processing page {page}/{TOTAL_PAGES}...")
        
        html_content = fetch_page(page)
        if html_content:
            page_links = extract_data_hrefs(html_content)
            all_links.extend(page_links)
            print(f"Found {len(page_links)} links on this page")
        
        # Be polite - add delay between requests
        time.sleep(1)
    
    # Save all links to file
    with open(OUTPUT_FILE, 'w') as f:
        for link in all_links:
            f.write(f"{link}\n")
    
    print(f"\nCompleted! Saved {len(all_links)} links to {OUTPUT_FILE}")

if __name__ == "__main__":
    main()

Processing page 1/100...
Found 140 links on this page
Processing page 2/100...
Found 140 links on this page
Processing page 3/100...
Found 140 links on this page
Processing page 4/100...
Found 140 links on this page
Processing page 5/100...
Found 140 links on this page
Processing page 6/100...
Found 140 links on this page
Processing page 7/100...
Found 140 links on this page
Processing page 8/100...
Found 140 links on this page
Processing page 9/100...
Found 140 links on this page
Processing page 10/100...
Found 140 links on this page
Processing page 11/100...
Found 140 links on this page
Processing page 12/100...
Found 140 links on this page
Processing page 13/100...
Found 140 links on this page
Processing page 14/100...
Found 140 links on this page
Processing page 15/100...
Found 140 links on this page
Processing page 16/100...
Found 140 links on this page
Processing page 17/100...
Found 140 links on this page
Processing page 18/100...
Found 140 links on this page
Processing page 19/

In [51]:
import requests
from bs4 import BeautifulSoup
import csv
import time

# File containing collected URLs
INPUT_FILE = "sustainalytics_links_all.txt"
OUTPUT_FILE = "esg_data.csv"

# Headers to mimic browser
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

def scrape_esg_data(url):
    """Scrape ESG data from a single Sustainalytics page"""
    try:
        response = requests.get(url, headers=HEADERS)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Extract data using CSS selectors
        data = {
            "url": url,
            "risk_rating_score": get_text(soup, '.risk-rating-score'),
            "risk_rating_assessment": get_text(soup, '.risk-rating-assessment'),
            "industry_group": get_text(soup, '.industry-group'),
            "identifier": get_text(soup, '.identifier'),
            "country": get_text(soup, '.country'),
            "timestamp": time.strftime("%Y-%m-%d %H:%M:%S")
        }
        return data
    except Exception as e:
        print(f"Error scraping {url}: {str(e)}")
        return None

def get_text(soup, selector):
    """Helper function to safely extract text"""
    element = soup.select_one(selector)
    return element.get_text(strip=True) if element else "N/A"

def main():
    # Read URLs from file
    with open(INPUT_FILE, 'r') as f:
        urls = [line.strip() for line in f if line.strip()]
    
    # Prepare CSV output
    with open(OUTPUT_FILE, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = [
            'url', 
            'risk_rating_score', 
            'risk_rating_assessment', 
            'industry_group', 
            'identifier', 
            'country',
            'timestamp'
        ]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        
        # Process each URL
        for i, url in enumerate(urls, 1):
            url="https://www.sustainalytics.com/esg-rating"+url
            print(f"Processing {i}/{len(urls)}: {url}")
            data = scrape_esg_data(url)
            if data:
                writer.writerow(data)
            
            # Be polite - add delay between requests
            time.sleep(1)
    
    print(f"\nCompleted! Data saved to {OUTPUT_FILE}")

if __name__ == "__main__":
    main()

Processing 1/3: https://www.sustainalytics.com/esg-rating/bystronic-ag/1008753208
Processing 2/3: https://www.sustainalytics.com/esg-rating/caregen-co-ltd/1050604623
Processing 3/3: https://www.sustainalytics.com/esg-rating/flowers-foods-inc/1008850361

Completed! Data saved to esg_data.csv


In [5]:
import pandas as pd # for data processing


In [20]:
df1 = pd.read_csv("esg_data1-5880.csv")
df2 = pd.read_csv("esg_data5881-10961.csv")
df3 = pd.read_csv("esg_data10962-14000.csv")
df4 = pd.read_csv("esg_data14000-14003.csv")
frames = [df1,df2,df3,df4]
df = pd.concat(frames, ignore_index=True)

In [21]:
df.head()

,url,risk_rating_score,risk_rating_assessment,industry_group,identifier,country,timestamp
0,https://www.sustainalytics.com/esg-rating/1-80...,28.9,Medium Risk,Retailing,NAS:FLWS,United States of America,2025-04-18 13:53:38
1,https://www.sustainalytics.com/esg-rating/1-1-...,27.7,Medium Risk,Telecommunication Services,ETR:1U1,Germany,2025-04-18 13:53:41
2,https://www.sustainalytics.com/esg-rating/10x-...,22.5,Medium Risk,Pharmaceuticals,NAS:TXG,United States of America,2025-04-18 13:53:43
3,https://www.sustainalytics.com/esg-rating/11-b...,16.3,Low Risk,Software & Services,WAR:11B,Poland,2025-04-18 13:53:46
4,https://www.sustainalytics.com/esg-rating/1st-...,35.0,High Risk,Banks,NAS:SRCE,United States of America,2025-04-18 13:53:48


In [22]:
df.tail()

,url,risk_rating_score,risk_rating_assessment,industry_group,identifier,country,timestamp
13995,https://www.sustainalytics.com/esg-rating/zoom...,17.2,Low Risk,Software & Services,NAS:ZI,United States of America,2025-04-19 01:38:52
13996,https://www.sustainalytics.com/esg-rating/zoom...,37.3,High Risk,Machinery,HKG:1157,China,2025-04-19 01:38:54
13997,https://www.sustainalytics.com/esg-rating/zyme...,29.0,Medium Risk,Pharmaceuticals,NAS:ZYME,United States of America,2025-04-19 09:09:15
13998,https://www.sustainalytics.com/esg-rating/zyne...,32.0,High Risk,Healthcare,NAS:ZYXI,United States of America,2025-04-19 09:09:16
13999,https://www.sustainalytics.com/esg-rating/zyxe...,21.6,Medium Risk,Technology Hardware,TAI:3704,Taiwan,2025-04-19 09:09:18


In [23]:
len(df)

14000

In [17]:
#len(df)
df_test = pd.concat(frames, ignore_index=True)
#df.shape[0]

In [25]:
def remove_prefix(url):
    prefix = "https://www.sustainalytics.com/esg-rating"
    if url.startswith(prefix):
        return url[len(prefix):]
    else:
        None

In [33]:
df1=df['url'].apply(remove_prefix)

In [39]:
df2=pd.read_csv("sustainalytics_links.txt")
df2.head()

,/1-800-flowers-com-inc/1007902331
0,/1-1-ag/1264207944
1,/10x-genomics-inc/2000259050
2,/11-bit-studios-sa/2002992103
3,/1st-source-corp/1008054650
4,/2020-bulkers-ltd/2005551813


In [41]:
df1.columns=['url']
df2.columns=['url']

In [46]:
merged_df = pd.merge(df2, df1, on='url', how='left', indicator=True)
missing_in_df1 = merged_df[merged_df['_merge'] == 'left_only'][['url']]

In [47]:
missing_in_df1

,url
1959,/bystronic-ag/1008753208
2121,/caregen-co-ltd/1050604623
4449,/flowers-foods-inc/1008850361


In [52]:
df_missed = pd.read_csv("esg_data_missing.csv")

In [54]:
df = pd.concat([df, df_missed], ignore_index=True)

In [55]:
df.head()

,url,risk_rating_score,risk_rating_assessment,industry_group,identifier,country,timestamp
0,https://www.sustainalytics.com/esg-rating/1-80...,28.9,Medium Risk,Retailing,NAS:FLWS,United States of America,2025-04-18 13:53:38
1,https://www.sustainalytics.com/esg-rating/1-1-...,27.7,Medium Risk,Telecommunication Services,ETR:1U1,Germany,2025-04-18 13:53:41
2,https://www.sustainalytics.com/esg-rating/10x-...,22.5,Medium Risk,Pharmaceuticals,NAS:TXG,United States of America,2025-04-18 13:53:43
3,https://www.sustainalytics.com/esg-rating/11-b...,16.3,Low Risk,Software & Services,WAR:11B,Poland,2025-04-18 13:53:46
4,https://www.sustainalytics.com/esg-rating/1st-...,35.0,High Risk,Banks,NAS:SRCE,United States of America,2025-04-18 13:53:48


In [58]:
df.to_csv("esg_data_org.csv", index=False)

In [59]:
df['company_name']=df['url'].apply(remove_prefix)

In [61]:
"/1-800-flowers-com-inc/1007902331".split("/")[1]

'1-800-flowers-com-inc'

In [62]:
def get_company_name(url):
    return url.split("/")[1]

In [63]:
df['company_name']=df['company_name'].apply(get_company_name)

In [65]:
df.drop(columns=['url','timestamp'], inplace=True)


In [67]:
df.to_csv("esg_data_for_project.csv", index=False)